In [41]:
#@markdown Setup dependencies (Colab only)
%%capture
try:
    import papyrus_scripts
except:
    !pip uninstall papyrus-scripts -y
    !pip install rdkit-pypi
    !pip install https://github.com/OlivierBeq/Papyrus-scripts/tarball/master --no-cache-dir
    get_ipython().kernel.do_shutdown(True)

# Simple examples: Using Papyrus scripts

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OlivierBeq/Papyrus-scripts/blob/master/notebook_examples/simple_examples.ipynb)

Herein it is assumed that the Papyrus <a href="https://doi.org/10.4121/16896406">bioactivity data</a> hosted on 4TU or Google Drive was ***NOT*** dowloaded.

In [42]:
%%html
<style>
table {align:left;display:block}
</style>

## Download the data


One can easily download (part of) the Papyrus data using *papyrus* command (*download* subcommand).<br/>
```
papyrus download --version latest -s without -S -d mold2 -d unirep
```



It can also be carried out programatically with the following.

In [43]:
from papyrus_scripts.download import download_papyrus

The default behaviour is to:
- download the curated 2D data (*nostereo* argument)
- omit the not curated 3D data (*stereo*)
- omit molecular structures (*structures*)
- download all molecular/sequence descriptors (*descriptors*)

Let's download the 2D curated data with molecular structures and only descriptors used below (total of 2.80 GB).

In [44]:
download_papyrus(version='latest', structures=True, descriptors=['mold2', 'unirep'])

Latest version: 05.7
########## DISCLAIMER ##########
You are downloading the high-quality Papyrus++ dataset.
Should you want to access the entire, though of lower quality, Papyrus dataset,
look into additional switches of this command.
################################
Number of files to be downloaded: 7
Total size: 2.26GB


At the time of writing (Apr. 26th, 2022) the latest version available is 05.5.

A custom directory to donwload the data to can be indicated with the *outdir* argument.

## Reading Papyrus files

Functions can be found under *papyrus_scripts.reader* to facilitate the dataset being read from disk.

In [45]:
from papyrus_scripts.reader import read_papyrus, read_protein_set

### Bioactivity data

Let's first read the bioactivity data.

We will use the *read_papyrus* function to read the bioactivity data as a pandas dataframe. <br/>
Let us first demonstrate the use of the function on systems with limited RAM (less than 50GB).

We first ensure to read the standardized data without stereochemistry in chunks of ten thousand lines.<br/>
Additionally we ensure the *source_path* is that given to the *download_papyrus* function above through the *outdir* argument.

In [46]:
sample_data = read_papyrus(is3d=False, chunksize=10000, source_path=None)

The return value is an iterator of Pandas dataframes of maximum ten thousand rows each.<br/>
Let's extract the first chunk as a pandas dataframe and have a look at few rows.

In [47]:
chunk1 = next(sample_data)
chunk1.head()

,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,type_other,Activity_class,relation,pchembl_value,pchembl_value_Mean,pchembl_value_StdDev,pchembl_value_SEM,pchembl_value_N,pchembl_value_Median,pchembl_value_MAD
0,AAAAEENPAALFRN_on_P49654_WT,High,ChEMBL34,ChEMBL:CHEMBL492934,COc1cc(C(C)C)c(Oc2cnc(NCCS(C)(=O)=O)nc2N)cc1I,AAAAEENPAALFRN,AAAAEENPAALFRN-UHFFFAOYSA-N,InChI=1S/C17H23IN4O4S/c1-10(2)11-7-14(25-3)12(...,"AuxInfo=1/1/N:7,8,1,19,16,17,4,25,12,6,5,26,9,...",P49654_WT,...,NaN,NaN,=,7.8,7.800,0.000000,0.000,1.0,7.800,0.000
1,AAAAZQPHATYWOK_on_P00533_WT,High,ChEMBL34,ChEMBL:CHEMBL175513;Sharma2016.compound.4277046,CCOc1cc2ncc(C#N)c(Nc3ccc(OCc4nc5ccccc5s4)c(Cl)...,AAAAZQPHATYWOK,AAAAZQPHATYWOK-UHFFFAOYSA-N,InChI=1S/C32H29ClN6O3S/c1-4-41-28-16-25-22(15-...,"AuxInfo=1/1/N:1,42,43,2,24,25,39,23,26,38,15,1...",P00533_WT,...,NaN,NaN,=,6.73;6.726,6.728,0.002828,0.002,2.0,6.728,0.002
2,AAAAZQPHATYWOK_on_P04626_WT,High,ChEMBL34,ChEMBL:CHEMBL175513,CCOc1cc2ncc(C#N)c(Nc3ccc(OCc4nc5ccccc5s4)c(Cl)...,AAAAZQPHATYWOK,AAAAZQPHATYWOK-UHFFFAOYSA-N,InChI=1S/C32H29ClN6O3S/c1-4-41-28-16-25-22(15-...,"AuxInfo=1/1/N:1,42,43,2,24,25,39,23,26,38,15,1...",P04626_WT,...,NaN,NaN,=,7.19,7.190,0.000000,0.000,1.0,7.190,0.000
3,AAACBXVBBDAYRQ_on_Q5S007_WT,High,ChEMBL34,ChEMBL:CHEMBL4067228,O=C(Nc1cccnc1)c1cc(Cl)ccc1OCc1ccccc1,AAACBXVBBDAYRQ,AAACBXVBBDAYRQ-UHFFFAOYSA-N,InChI=1S/C19H15ClN2O2/c20-15-8-9-18(24-13-14-5...,"AuxInfo=1/1/N:22,21,23,6,20,24,5,14,15,7,11,9,...",Q5S007_WT,...,NaN,NaN,=,7.8,7.800,0.000000,0.000,1.0,7.800,0.000
4,AAACGYYPWMUUFL_on_O75116_WT,High,Sharma2016,Sharma2016.compound.10060010,COC(=O)CC(NC(=O)c1ccc(-c2cn[nH]c2)c(C)c1)c1ccccc1,AAACGYYPWMUUFL,AAACGYYPWMUUFL-UHFFFAOYSA-N,InChI=1S/C21H21N3O3/c1-14-10-16(8-9-18(14)17-1...,"AuxInfo=1/1/N:20,1,25,24,26,23,27,11,12,21,5,1...",O75116_WT,...,NaN,NaN,=,7.154,7.154,0.000000,0.000,1.0,7.154,0.000


If you are sure your hardware can handle loading all the data, then you can drop *chunksize*.<br/>
Then the return value is a pandas dataframe.

Below, we will show how to use:
  - a pandas dataframe (by calling our methods on *chunk1*)
  - an iterator of pandas dataframes (by calling our methods on *sample_data*)

### Protein target data

But for now let's focus on protein data:<br/>
Information about the protein targets is available from a different file and can be loaded as easily as was demonstrated above.<br/>
This file being very limited in size, chunking is not needed.

In [48]:
protein_data = read_protein_set(source_path=None)
protein_data.head()

,target_id,UniProtID,Status,Organism,Classification,Length,Sequence,TID
0,P47747_WT,HRH2_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,359,MAFNGTVPSFCMDFTVYKVTISVILIILILVTVAGNVVVCLAVGLN...,ChEMBL:CHEMBL2882
1,B0FL73_WT,B0FL73_CAVPO,unreviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,467,MGAGVLALGASEPCNLSSTAPLPDGAATAARLLVPASPPASLLPPT...,ChEMBL:CHEMBL5471
2,Q8K4Z4_WT,ADRB2_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,418,MGHLGNGSDFLLAPNASHAPDHNVTRERDEAWVVGMAIVMSLIVLA...,ChEMBL:CHEMBL5414
3,P97266_WT,OPRM_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,98,YTKMKTATNIYIFNLALADALATSTLPFQSVNYLMGTWPFGTILCK...,ChEMBL:CHEMBL4354;ChEMBL:CHEMBL2111465
4,P41144_WT,OPRK_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,380,MGRRRQGPAQPASELPARNACLLPNGSAWLPGWAEPDGNGSAGPQD...,ChEMBL:CHEMBL3952;ChEMBL:CHEMBL2111465


## Filtering Papyrus

The data contained in the dataset can be filtered very easily using functions under *papyrus_scripts.preprocess*.<br/>
All filtering functions start with ***keep_***.

In [49]:
from papyrus_scripts.preprocess import (keep_quality, keep_source, keep_type,
                                        keep_organism, keep_accession, keep_protein_class,
                                        keep_match, keep_contains
                                       )

**The strength of the Papyrus scripts is that the data can be filtered whether chunked or not.** The only difference:
  - when using chunked data, call *consume_chunks* once all filters are applied to reconstiture a pandas dataframe

In addition to the 8 **keep_** functions above, one can filter compounds similar to a reference using the *keep_similar* and *keep_substructures* functions (see [advanced_querying.ipynb](https://github.com/OlivierBeq/Papyrus-scripts/blob/master/notebook_examples/advanced_querying.ipynb)).

### Filtering pandas dataframes

Let's first keep the data with quality 'medium' and above (namely 'high' and 'medium').

In [50]:
filter1 = keep_quality(data=chunk1, min_quality='medium')

<u>Using <a href="https://www.ebi.ac.uk/chembl/visualise/">ChEMBL's protein target tree</a> is encouraged for this part.</u><br/>
<br/>
We will then filter out any protein not belonging to these two classes:
* Ligand-gated ion channels
* SLC superfamily of solute carriers

For this filter, passing protein information is required (the same applies for *keep_organism* and *keep_accession*).

In [51]:
filter2 = keep_protein_class(data=filter1, protein_data=protein_data, classes=[{'l2': 'Ligand-gated ion channels'}, {'l3': 'SLC superfamily of solute carriers'}])

We now keep only K<sub>i</sub> and K<sub>D</sub> data.<br/>
Here we will pass filter1 to the next *keep_* funtion.

In [52]:
filter3 = keep_type(data=filter2, activity_types=['Ki', 'KD'], njobs=1)

We finally keep only human and rat data (protein information is also required here).

In [53]:
filter4 = keep_organism(data=filter3, protein_data=protein_data, organism=['Human', 'Rat'], generic_regex=True)

Let us have a look at the filtered data.

In [54]:
filter4.head()

,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,relation,pchembl_value,pchembl_value_Mean,pchembl_value_StdDev,pchembl_value_SEM,pchembl_value_N,pchembl_value_Median,pchembl_value_MAD,Classification,Organism
0,AAEKULYONKUBOZ_on_P23977_WT,High,ChEMBL34,ChEMBL:CHEMBL14144,CN1C2CCC1C(C(=O)Oc1ccccc1)C(c1ccc(Cl)cc1)C2,AAEKULYONKUBOZ,AAEKULYONKUBOZ-UHFFFAOYSA-N,InChI=1S/C21H22ClNO2/c1-23-16-11-12-19(23)20(2...,"AuxInfo=1/0/N:1,14,13,15,12,16,19,24,20,23,4,5...",P23977_WT,...,=,8.28,8.28,0.0,0.0,1.0,8.28,0.0,Transporter->Electrochemical transporter->SLC ...,Rattus norvegicus (Rat)
1,AAEKULYONKUBOZ_on_P31645_WT,High,ChEMBL34,ChEMBL:CHEMBL14144,CN1C2CCC1C(C(=O)Oc1ccccc1)C(c1ccc(Cl)cc1)C2,AAEKULYONKUBOZ,AAEKULYONKUBOZ-UHFFFAOYSA-N,InChI=1S/C21H22ClNO2/c1-23-16-11-12-19(23)20(2...,"AuxInfo=1/0/N:1,14,13,15,12,16,19,24,20,23,4,5...",P31645_WT,...,=,6.41,6.41,0.0,0.0,1.0,6.41,0.0,Transporter->Electrochemical transporter->SLC ...,Homo sapiens (Human)
2,AAEKULYONKUBOZ_on_Q9WTR4_WT,High,ChEMBL34,ChEMBL:CHEMBL14144,CN1C2CCC1C(C(=O)Oc1ccccc1)C(c1ccc(Cl)cc1)C2,AAEKULYONKUBOZ,AAEKULYONKUBOZ-UHFFFAOYSA-N,InChI=1S/C21H22ClNO2/c1-23-16-11-12-19(23)20(2...,"AuxInfo=1/0/N:1,14,13,15,12,16,19,24,20,23,4,5...",Q9WTR4_WT,...,=,6.62,6.62,0.0,0.0,1.0,6.62,0.0,Transporter->Electrochemical transporter->SLC ...,Rattus norvegicus (Rat)
3,AAHLCKVWZXFMPB_on_P23975_WT,High,ChEMBL34,ChEMBL:CHEMBL3323101,CN1Cc2ccccc2C(c2ccc3[nH]ccc3c2)C1,AAHLCKVWZXFMPB,AAHLCKVWZXFMPB-UHFFFAOYSA-N,InChI=1S/C18H18N2/c1-20-11-15-4-2-3-5-16(15)17...,"AuxInfo=1/0/N:1,6,7,5,8,12,13,17,16,19,3,20,11...",P23975_WT,...,=,8.24,8.24,0.0,0.0,1.0,8.24,0.0,Transporter->Electrochemical transporter->SLC ...,Homo sapiens (Human)
4,AAHLCKVWZXFMPB_on_P31645_WT,High,ChEMBL34,ChEMBL:CHEMBL3323101,CN1Cc2ccccc2C(c2ccc3[nH]ccc3c2)C1,AAHLCKVWZXFMPB,AAHLCKVWZXFMPB-UHFFFAOYSA-N,InChI=1S/C18H18N2/c1-20-11-15-4-2-3-5-16(15)17...,"AuxInfo=1/0/N:1,6,7,5,8,12,13,17,16,19,3,20,11...",P31645_WT,...,=,6.86,6.86,0.0,0.0,1.0,6.86,0.0,Transporter->Electrochemical transporter->SLC ...,Homo sapiens (Human)


In [55]:
print(f'Number of activity points: {filter4.shape[0]}')

Number of activity points: 114


Remember that this result comes from only the first chunk of the entire dataset.

One can now save this dataframe like any other pandas object.

### Filtering iterators of dataframes

Now that the filtering capacity of the Papyrus scripts have been demonstrated for entire dataframes, we can try with chunked iterators.

Let's first reinstanciate sample data. This time we will use a chunk size of 1,000,000.

In [56]:
sample_data = read_papyrus(is3d=False, chunksize=1000000, source_path=None)

For this will will go through the same filters as above but iterate over the entire dataset.

In [57]:
filter1_it = keep_quality(data=sample_data, min_quality='medium')
filter2_it = keep_protein_class(data=filter1_it, protein_data=protein_data, classes=[{'l2': 'Ligand-gated ion channels'}, {'l3': 'SLC superfamily of solute carriers'}])
filter3_it = keep_type(data=filter2_it, activity_types=['Ki', 'KD'])
filter4_it = keep_organism(data=filter3_it, protein_data=protein_data, organism=['Human', 'Rat'], generic_regex=True)

The filters do not get applied directly on chunked iterators and one can easily check that *filter4_it* is not a pandas dataframe.

In [58]:
filter4_it

<generator object _chunked_keep_organism at 0x7ab6d0165770>

To apply the filters on the entire iterator, one needs to call *consume_chunks*.<br/>
This function can be found under *papyrus_scripts.preprocess* just like the *keep_* functions used for filtering.

In [59]:
from papyrus_scripts.preprocess import consume_chunks

In order to follow progress of the filtering process, one needs to pass the total number of chunks the filters will go through.<br/>
$Total = \displaystyle \Bigl \lceil\frac{Size_{dataset}}{chunksize}\Bigl \rceil $<br/>

In version 05.5 of the Papyrus dataset the number of compound-protein activity points depends on whether stereochemistry is used or not **(remember we discourage its usage)**.<br/>


| Stereochemistry | Size of dataset |
| :--- | :---: |
| Without | 59,775,912 |
| With (strongly discouraged) | 61,097,228 |

In this example $Total = \displaystyle \Bigl \lceil \frac{59,775,912}{1,000,000}\Bigl \rceil = 60 $<br/>

In [60]:
filtered_data = consume_chunks(filter4_it, progress=True, total=60)

  0%|          | 0/60 [00:00<?, ?it/s]

Although this may take up to 30 minutes to filter the entire dataset, this is the ideal way to work with this dataset on laptops.

In [61]:
print(f'Number of activity points: {filtered_data.shape[0]}')

Number of activity points: 7528


We hope these simple examples demonstrated how the Papyrus data can easily be filtered.
Let's now focus on the modelling

## Modelling the bioactivity data

The Papyrus scripts allow for both quantitative structure-activity relationship (QSAR) and proteochemometrics (PCM) modelling.<br/>
All functions related to modelling can be found under *papyrus_scripts.modelling*.

**Disclaimer:**<br/>
For now, only precomputed molecular descriptors can be used, preventing the use of models outside of Papyrus.<br/>
This major flaw will be soon fixed.

### QSAR models

In [62]:
pip install scikit-learn==1.2

In [63]:
from papyrus_scripts.modelling import qsar
import xgboost

Let us first restrict the data that we just extracted from Papyrus to the human serotonin receptor (accession P31645).

In [73]:
sample_data = read_papyrus(is3d=False, chunksize=1000000, source_path=None)
filter1_it = keep_accession(sample_data, 'P01116')
filter2_it = keep_quality(data=filter1_it, min_quality='medium')
filter3_it = keep_type(data=filter2_it, activity_types=['Ki', 'KD'])

In [74]:
SLC6A4_data = consume_chunks(filter3_it, total=60)

  0%|          | 0/60 [00:00<?, ?it/s]

In [84]:
print(SLC6A4_data.head(5))

                       Activity_ID Quality    source                   CID  \
9863   AIRAONOHNMWJKZ_on_P01116_WT    High  ChEMBL34  ChEMBL:CHEMBL5191782   
27905  AYYURYDKBCGXDD_on_P01116_WT    High  ChEMBL34  ChEMBL:CHEMBL3982443   
28135  AZFJASMBZUATSI_on_P01116_WT    High  ChEMBL34  ChEMBL:CHEMBL3932440   
56645  BYJMARHSXWNYIZ_on_P01116_WT    High  ChEMBL34  ChEMBL:CHEMBL5085395   
68904  CKAMBYUZKWQCKJ_on_P01116_WT    High  ChEMBL34  ChEMBL:CHEMBL5218642   

                                                  SMILES    connectivity  \
9863   COc1ccc(Cl)cc1-c1c(Cl)cc2c(N3CCN(C(=O)c4ccc([N...  AIRAONOHNMWJKZ   
27905  Nc1nc(=O)c2ncn(C3OC(COP(=O)(O)CS(=O)(=O)NCCNC(...  AYYURYDKBCGXDD   
28135  Nc1nc(=O)c2ncn(C3OC(COP(=O)(O)C(F)(F)P(=O)(O)O...  AZFJASMBZUATSI   
56645  CN1CCN(c2nc(OCC3CCCN3C)nc3c(F)c(-c4cc(O)cc5ccc...  BYJMARHSXWNYIZ   
68904  C=CC(=O)N1CCN(c2cc(-c3noc(C4(C)CCCc5sc(N)c(C#N...  CKAMBYUZKWQCKJ   

                          InChIKey  \
9863   AIRAONOHNMWJKZ-UHFFFAOYSA-N  

In [87]:
# 检查列名
print(SLC6A4_data.columns)

# 如果列名是 'Index' 或其他形式，重命名为 'index'
if 'Index' in SLC6A4_data.columns:
    SLC6A4_data.rename(columns={'Index': 'index'}, inplace=True)

# 手动设置索引
SLC6A4_data.set_index('index', inplace=True)


Index(['Activity_ID', 'Quality', 'source', 'CID', 'SMILES', 'connectivity',
       'InChIKey', 'InChI', 'InChI_AuxInfo', 'target_id', 'TID', 'accession',
       'Protein_Type', 'AID', 'doc_id', 'Year', 'all_doc_ids', 'all_years',
       'type_IC50', 'type_EC50', 'type_KD', 'type_Ki', 'type_other',
       'Activity_class', 'relation', 'pchembl_value', 'pchembl_value_Mean',
       'pchembl_value_StdDev', 'pchembl_value_SEM', 'pchembl_value_N',
       'pchembl_value_Median', 'pchembl_value_MAD', 'index'],
      dtype='object')


In [77]:
import numpy as np

def process_pchembl_value(value):
    if isinstance(value, str) and ';' in value:
        values = list(map(float, value.split(';')))
        return np.mean(values)
    return float(value)

SLC6A4_data['pchembl_value_Mean'] = SLC6A4_data['pchembl_value_Mean'].apply(process_pchembl_value)


In [79]:
SLC6A4_data.dropna(subset=['pchembl_value_Mean'], inplace=True)  # 删除目标变量缺失的行


In [82]:
SLC6A4_data['index'] = range(len(SLC6A4_data))


In [89]:
print(SLC6A4_data.head())
print(SLC6A4_data.columns)


                       Activity_ID Quality    source                   CID  \
index                                                                        
0      AIRAONOHNMWJKZ_on_P01116_WT    High  ChEMBL34  ChEMBL:CHEMBL5191782   
1      AYYURYDKBCGXDD_on_P01116_WT    High  ChEMBL34  ChEMBL:CHEMBL3982443   
2      AZFJASMBZUATSI_on_P01116_WT    High  ChEMBL34  ChEMBL:CHEMBL3932440   
3      BYJMARHSXWNYIZ_on_P01116_WT    High  ChEMBL34  ChEMBL:CHEMBL5085395   
4      CKAMBYUZKWQCKJ_on_P01116_WT    High  ChEMBL34  ChEMBL:CHEMBL5218642   

                                                  SMILES    connectivity  \
index                                                                      
0      COc1ccc(Cl)cc1-c1c(Cl)cc2c(N3CCN(C(=O)c4ccc([N...  AIRAONOHNMWJKZ   
1      Nc1nc(=O)c2ncn(C3OC(COP(=O)(O)CS(=O)(=O)NCCNC(...  AYYURYDKBCGXDD   
2      Nc1nc(=O)c2ncn(C3OC(COP(=O)(O)C(F)(F)P(=O)(O)O...  AZFJASMBZUATSI   
3      CN1CCN(c2nc(OCC3CCCN3C)nc3c(F)c(-c4cc(O)cc5ccc...  BYJMARHSXWNYIZ 

In [88]:
# 创建数据的副本
data_copy = deepcopy(SLC6A4_data)


NameError: name 'deepcopy' is not defined

We will first create a regression model predicting the average pActivity values of a compound-target pair (i.e. *pchembl_value_Mean*).<br/>
Let's ensure the version below is that given to the *download_papyrus* function above.

In [80]:
reg_model = xgboost.XGBRegressor(verbosity=0)

In [86]:
reg_results, trained_reg_model = qsar(
    data=SLC6A4_data,
    endpoint='pchembl_value_Mean',
    verbose=True
)

Loading molecular descriptors: 0it [00:00, ?it/s]

Loading molecular descriptors: 0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

KeyError: "None of ['index'] are in the columns"

In [85]:
reg_results, trained_reg_model = qsar(
    data=SLC6A4_data,
    version='latest',
    endpoint='pchembl_value_Mean',
    num_points=20,  # 调整为较小的值
    delta_activity=1,  # 调整为较小的值
    descriptors='mold2',
    descriptor_chunksize=50000,
    activity_threshold=5.0,  # 根据数据分布调整
    model=reg_model,
    folds=5,
    stratify=False,
    split_by='Year',
    split_year=2013,
    test_set_size=0.20,  # 调整为较小的值
    cluster_method=None,
    custom_groups=None,
    random_state=1234,
    verbose=True
)

Loading molecular descriptors: 0it [00:00, ?it/s]

Loading molecular descriptors: 0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

KeyError: "None of ['index'] are in the columns"

In [ ]:
reg_results

When looking at average R<sup>2</sup>, performance over cross-validation is correct but the model show very little capacity to predict the temporally split test set.

To train a classifier, all that is needed is to change the type of model.

In [ ]:
cls_model = xgboost.XGBClassifier(verbosity=0)

In [ ]:
cls_results, trained_cls_model = qsar(data=SLC6A4_data,
                                      version='latest',
                                      endpoint='pchembl_value_Mean',
                                      num_points=30,
                                      delta_activity=2,
                                      descriptors='mold2',
                                      descriptor_chunksize=50000,
                                      activity_threshold=6.5,
                                      model=cls_model,
                                      folds=5,
                                      stratify=False,
                                      split_by='Year',
                                      split_year=2013,
                                      test_set_size=0.30,
                                      cluster_method=None,
                                      custom_groups=None,
                                      random_state=1234,
                                      verbose=True)

In [ ]:
cls_results

Looking at the active to inactive ratio (i.e. A:N) one can clearly identify the reason of this low prediction performance over the test set.<br/>
Oversampling and/or undersampling techniques could help the model better identify the boundary between actives and inactives in the mmecular descriptor space.<br/>
However the use of such techniques is not the focus here.

### PCM models

In [ ]:
from papyrus_scripts.modelling import pcm

Let us see if including Rat data improves the quality of the model.

In [ ]:
sample_data = read_papyrus(is3d=False, chunksize=1000000, source_path=None)
filter1_it = keep_accession(sample_data, ['P31645', 'P31652'])
filter2_it = keep_quality(data=filter1_it, min_quality='medium')
filter3_it = keep_type(data=filter2_it, activity_types=['Ki', 'KD'])

In [ ]:
SLC6A4_human_rat = consume_chunks(filter3_it, total=60)

Let's ensure the version below is that given to the *download_papyrus* function above.

In [ ]:
pcm_reg_model = xgboost.XGBRegressor(verbosity=0)

In [ ]:
pcm_reg_results, pcm_reg_trained_model = pcm(data=SLC6A4_human_rat,
                                             version='latest',
                                             endpoint='pchembl_value_Mean',
                                             num_points=30,
                                             delta_activity=2,
                                             mol_descriptors='mold2',
                                             mol_descriptor_chunksize=50000,
                                             prot_descriptors='unirep',
                                             prot_descriptor_chunksize=50000,
                                             activity_threshold=6.5,
                                             model=pcm_reg_model,
                                             folds=5,
                                             stratify=False,
                                             split_by='Year',
                                             split_year=2013,
                                             test_set_size=0.30,
                                             cluster_method=None,
                                             custom_groups=None,
                                             random_state=1234,
                                             verbose=True)

In [ ]:
pcm_reg_results

As with QSAR models, training a classifier is a matter of changing the underlying model to be used.

In [ ]:
pcm_cls_model = xgboost.XGBClassifier(verbosity=0)

In [ ]:
pcm_cls_results, pcm_cls_trained_model = pcm(data=SLC6A4_human_rat,
                                             version='latest',
                                             endpoint='pchembl_value_Mean',
                                             num_points=30,
                                             delta_activity=2,
                                             mol_descriptors='mold2',
                                             mol_descriptor_chunksize=50000,
                                             prot_descriptors='unirep',
                                             prot_descriptor_chunksize=50000,
                                             activity_threshold=6.5,
                                             model=pcm_cls_model,
                                             folds=5,
                                             stratify=False,
                                             split_by='Year',
                                             split_year=2013,
                                             test_set_size=0.30,
                                             cluster_method=None,
                                             custom_groups=None,
                                             random_state=1234,
                                             verbose=True)

In [ ]:
pcm_cls_results